In [11]:
import pandas as pd
import os.path as op
import yaml
import os
import glob
from collections import Counter

from scgc import run

In [3]:
config = yaml.load(open("../vcprofiler.yaml"))

In [4]:
config

{'databases': {'blast': {'djr': '/mnt/scgc/simon/simonsproject/jb_vs_test/viral_dbs/djr/djr_proteins.fasta',
   'imgvr': {'nucl': '/mnt/scgc/simon/simonsproject/jb_vs_test/viral_dbs/IMGVR/IMGVR_mVCs_nucleotides.fna',
    'prot': '/mnt/scgc/simon/simonsproject/jb_vs_test/viral_dbs/IMGVR/IMGVR_mVCs_proteins.faa'},
   'ssdna_virs': '/mnt/scgc/simon/simonsproject/jb_vs_test/viral_dbs/ssdna/ssDNA_viruses.fna',
   'vog': '/mnt/scgc/simon/simonsproject/jb_vs_test/viral_dbs/VOG/vog_proteins_nr.fasta'},
  'hmm': {'vog': '/mnt/scgc/simon/simonsproject/jb_vs_test/viral_dbs/VOG/hmm/vog_hmm'},
  'taxonomy': {'vog': '/mnt/scgc/simon/simonsproject/jb_vs_test/viral_dbs/VOG/vog.lca.tsv'}}}

In [19]:
totest = '/mnt/scgc_nfs/lab/julia/notebooks/VCProfiler/test_data/bats248_vspos_subset.fasta'
db = config['databases']['hmm']['vog']
outtbl = "/mnt/scgc_nfs/lab/julia/notebooks/VCProfiler/test_data/hmm_out_test_tbl.out"
outfile = "/mnt/scgc_nfs/lab/julia/notebooks/VCProfiler/test_data/hmm_out_test.out"

```
julia at c1 in /mnt/scgc/simon/simonsproject/jb_vs_test/viral_dbs/VOG/hmm
$ hmmscan -h
# hmmscan :: search sequence(s) against a profile database
# HMMER 3.1b2 (February 2015); http://hmmer.org/
# Copyright (C) 2015 Howard Hughes Medical Institute.
# Freely distributed under the GNU General Public License (GPLv3).
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
Usage: hmmscan [-options] <hmmdb> <seqfile>

Basic options:
  -h : show brief help on version and usage

Options controlling output:
  -o <f>           : direct output to file <f>, not stdout
  --tblout <f>     : save parseable table of per-sequence hits to file <f>
  --domtblout <f>  : save parseable table of per-domain hits to file <f>
  --pfamtblout <f> : save table of hits and domains to file, in Pfam format <f>
  --acc            : prefer accessions over names in output
  --noali          : don't output alignments, so output is smaller
  --notextw        : unlimit ASCII text output line width
  --textw <n>      : set max width of ASCII text output lines  [120]  (n>=120)

Options controlling reporting thresholds:
  -E <x>     : report models <= this E-value threshold in output  [10.0]  (x>0)
  -T <x>     : report models >= this score threshold in output
  --domE <x> : report domains <= this E-value threshold in output  [10.0]  (x>0)
  --domT <x> : report domains >= this score cutoff in output

Options controlling inclusion (significance) thresholds:
  --incE <x>    : consider models <= this E-value threshold as significant
  --incT <x>    : consider models >= this score threshold as significant
  --incdomE <x> : consider domains <= this E-value threshold as significant
  --incdomT <x> : consider domains >= this score threshold as significant

Options for model-specific thresholding:
  --cut_ga : use profile's GA gathering cutoffs to set all thresholding
  --cut_nc : use profile's NC noise cutoffs to set all thresholding
  --cut_tc : use profile's TC trusted cutoffs to set all thresholding

Options controlling acceleration heuristics:
  --max    : Turn all heuristic filters off (less speed, more power)
  --F1 <x> : MSV threshold: promote hits w/ P <= F1  [0.02]
  --F2 <x> : Vit threshold: promote hits w/ P <= F2  [1e-3]
  --F3 <x> : Fwd threshold: promote hits w/ P <= F3  [1e-5]
  --nobias : turn off composition bias filter

Other expert options:
  --nonull2     : turn off biased composition score corrections
  -Z <x>        : set # of comparisons done, for E-value calculation
  --domZ <x>    : set # of significant seqs, for domain E-value calculation
  --seed <n>    : set RNG seed to <n> (if 0: one-time arbitrary seed)  [42]
  --qformat <s> : assert input <seqfile> is in format <s>: no autodetection
  --daemon      : run program as a daemon
  --cpu <n>     : number of parallel CPU workers to use for multithreads```

In [ ]:
def run_hmmscan(infasta, db, outtbl, threads=1):
    if op.exists(outtbl):
        return outtbl
    cmd = "hmmscan --tblout {outtbl} --cpu {threads} {db} {infasta}".format(outtbl=outtbl, db=db,                                                                             infasta=infasta, threads=threads)
    run(cmd)
    return outtbl

In [35]:
cmd = "hmmscan --tblout {outtbl} -o outfile --cpu 4 {db} {totest}".format(outtbl=outtbl, db=db, totest=totest)

In [13]:
!module load hmmer

In [36]:
run(cmd)

In [37]:
!head {outtbl} -n100

#                                                               --- full sequence ---- --- best 1 domain ---- --- domain number estimation ----
# target name        accession  query name           accession    E-value  score  bias   E-value  score  bias   exp reg clu  ov env dom rep inc description of target
#------------------- ---------- -------------------- ---------- --------- ------ ----- --------- ------ -----   --- --- --- --- --- --- --- --- ---------------------
VOG00597             -          AG-359-N20_NODE_1_2  -            2.1e-10   40.9   6.7   2.2e-10   40.9   6.7   1.0   1   0   0   1   1   1   1 -
VOG02633             -          AG-359-N20_NODE_1_2  -              0.081   13.7   0.2     0.089   13.5   0.2   1.2   1   0   0   1   1   1   0 -
VOG14276             -          AG-359-N20_NODE_1_2  -               0.14   12.5   0.3      0.62   10.4   0.0   2.1   3   0   0   3   3   3   0 -
VOG07748             -          AG-359-N20_NODE_1_2  -               0.26   11.8   1.3

In [97]:
def import_hmmscan_out(tbl):
    names = ['sseqid', 'accession', 'qseqid', 'accession', 'evalue', 'score', 'bias', 'd-evalue', 'd-score', 'd-bias', 'exp', 'reg', 'clu', 'ov', 'env', 'dom', 'rep', 'inc', 'description of target']
    df = pd.DataFrame(columns = names)
    with open(tbl) as ih:
        for i, l in enumerate(ih):
            if not l.startswith('#'):
                df.loc[i] = pd.Series(dict(zip(names, l.split())))
    return df

def hmmscan_besthits(df, max_eval=0.001):
    df['evalue'] = [float(i) for i in df['evalue']]
    df = df[df['evalue'] < max_eval]
    return df.sort_values(by=['qseqid','evalue'], ascending=True).drop_duplicates(subset='qseqid', keep='first')



In [98]:
dftop = hmmscan_besthits(df, max_eval=0.00001)

In [93]:
from vcprofiler import hits_per_contig, get_best_hits, import_blastp

hdf_contig_summary = hits_per_contig(dftop, 'vog_hmms')

Now compare BLAST to HMM results:

In [71]:
blast = '/mnt/scgc_nfs/lab/julia/notebooks/VCProfiler/test_data/bats248_vspos_subset_vs_vog_blastp.out'

In [117]:
bdf_top = get_best_hits(import_blastp(blast), min_align_pct=75, min_pct_id=35, min_bitscore=50) 

In [118]:
from vcprofiler import summarise_vog_blast

In [119]:
vogblast_contig_summary = pd.DataFrame(pd.Series(summarise_vog_blast(bdf_top).groupby('contig')['orf'].count(), name='vog_blast')).reset_index()

In [120]:
pd.merge(vogblast_contig_summary, hdf_contig_summary, on='contig', how='outer')

,contig,vog_blast,orfs_matching_vog_hmms
0,AG-359-N20_NODE_1,16,31
1,AG-359-O06_NODE_1,5,10
2,AG-359-O06_NODE_2,3,6
3,AG-359-O06_NODE_3,1,2
4,AG-359-O19_NODE_1,22,25
5,AG-359-O19_NODE_2,1,6
6,AG-359-O19_NODE_3,15,19
7,AG-359-O19_NODE_4,2,4
8,AG-359-O19_NODE_5,6,13


Lots more matches when using hmmer!

In [123]:
comp = summarise_vog_blast(bdf_top).merge(dftop, left_on='orf', right_on='qseqid', how='outer')[['orf','qseqid','vog','sseqid']].dropna()

In [136]:
print(len(comp))
print(len(comp[comp['vog'] == comp['sseqid']]))
print(round(abs(len(comp) - len(comp[comp['vog'] == comp['sseqid']])) / len(comp) * 100), "% mismatched between blast and hmmer", sep="")

69
65
6% mismatched between blast and hmmer


In [132]:
misses = comp[comp['vog'] != comp['sseqid']]['orf']

In [138]:
comp[comp['vog'] != comp['sseqid']]

,orf,qseqid,vog,sseqid
2,AG-359-N20_NODE_1_12,AG-359-N20_NODE_1_12,VOG00878,VOG21261
4,AG-359-N20_NODE_1_16,AG-359-N20_NODE_1_16,VOG09235,VOG00072
40,AG-359-O19_NODE_1_26,AG-359-O19_NODE_1_26,VOG00258,VOG00890
56,AG-359-O19_NODE_3_23,AG-359-O19_NODE_3_23,VOG00878,VOG21261


In [135]:
df[df['qseqid'].isin(misses)].sort_values(by=['qseqid', 'evalue'])

,sseqid,accession,qseqid,accession,evalue,score,bias,d-evalue,d-score,d-bias,exp,reg,clu,ov,env,dom,rep,inc,description of target
17,VOG21261,-,AG-359-N20_NODE_1_12,-,6.300000e-08,33.1,1.8,2.3e-07,31.3,1.2,2.2,2,1,0,2,2,2,1,-
22,VOG00072,-,AG-359-N20_NODE_1_16,-,4.800000e-78,263.8,0.0,6.2e-78,263.4,0.0,1.0,1,0,0,1,1,1,1,-
23,VOG09235,-,AG-359-N20_NODE_1_16,-,6.500000e-44,151.4,0.1,8.4e-44,151.0,0.1,1.0,1,0,0,1,1,1,1,-
24,VOG15795,-,AG-359-N20_NODE_1_16,-,5.400000e-02,12.9,0.0,0.08,12.4,0.0,1.2,1,0,0,1,1,1,0,-
25,VOG01188,-,AG-359-N20_NODE_1_16,-,7.800000e-02,12.3,0.0,0.11,11.8,0.0,1.1,1,0,0,1,1,1,0,-
26,VOG14992,-,AG-359-N20_NODE_1_16,-,2.000000e-01,11.9,0.1,0.39,11.0,0.1,1.4,1,0,0,1,1,1,0,-
222,VOG00890,-,AG-359-O19_NODE_1_26,-,2.100000e-25,90.3,6.3,2.3e-25,90.2,6.3,1.0,1,0,0,1,1,1,1,-
223,VOG00258,-,AG-359-O19_NODE_1_26,-,2.700000e-14,54.1,1.5,3e-14,53.9,1.5,1.0,1,0,0,1,1,1,1,-
224,VOG00190,-,AG-359-O19_NODE_1_26,-,2.400000e-08,34.9,3.5,2.4e-08,34.8,3.5,1.1,1,0,0,1,1,1,1,-
225,VOG01439,-,AG-359-O19_NODE_1_26,-,4.500000e-07,30.4,0.5,4.7e-07,30.4,0.5,1.1,1,0,0,1,1,1,1,-


Now to draw in bets hits and categories for the hmm output:

In [169]:
vdf = dftop[['qseqid','sseqid']].rename(columns={'sseqid':'vog','qseqid':'orf'})
voganns = pd.read_csv("/mnt/scgc/simon/simonsproject/jb_vs_test/viral_dbs/VOG/vog.annotations.tsv", sep="\t")
voglca = pd.read_csv("/mnt/scgc/simon/simonsproject/jb_vs_test/viral_dbs/VOG/vog.lca.tsv", skiprows=1, sep="\t", names=['vog','genomes_in_group_and_lca','genomes_total_in_lca','lca'])
vogvircats = pd.read_csv("/mnt/scgc/simon/simonsproject/jb_vs_test/viral_dbs/VOG/vog.virusonly.tsv.gz", sep="\t")

In [177]:
def vog_summary(vhmm_tops, 
                vog_annotations = "/mnt/scgc/simon/simonsproject/jb_vs_test/viral_dbs/VOG/vog.annotations.tsv", 
                vog_lca = "/mnt/scgc/simon/simonsproject/jb_vs_test/viral_dbs/VOG/vog.lca.tsv"):
    vhmm_tops = dftop[['qseqid','sseqid']].rename(columns={'sseqid':'vog','qseqid':'orf'})
    voganns = pd.read_csv(vog_annotations, sep="\t")
    voganns.rename(columns={'#GroupName':'vog','ConsensusFunctionalDescription':'func_desc'}, inplace=True)
    voglca = pd.read_csv(vog_lca, skiprows=1, sep="\t", names=['vog','genomes_in_group_and_lca','genomes_total_in_lca','lca'])
    summary = pd.merge(vdf, 
                           voganns, 
                           on='vog', 
                           how='left').merge(voglca, 
                                             on='vog', 
                                             how='left')
    summary['contig'] = ["_".join(i.split("_")[:-1]) for i in summary['orf']]
    summary['orf_num'] = [int(i.split("_")[-1]) for i in summary['orf']]
    summary = summary.sort_values(by=['contig','orf_num']).reset_index()
    return summary[['orf','contig', 'vog','func_desc','lca']]

In [183]:
vs = vog_summary(dftop)

In [164]:
voganns.rename(columns={'#GroupName':'vog'}, inplace=True)

In [168]:
vog_summary = pd.merge(vdf, voganns, on='vog', how='left').merge(voglca, on='vog', how='left')[['orf','vog','ConsensusFunctionalDescription','lca']]

,orf,vog,ConsensusFunctionalDescription,lca
0,AG-359-N20_NODE_1_10,VOG10983,REFSEQ hypothetical protein,"Viruses;dsDNA viruses, no RNA stage;Caudovirales"
1,AG-359-N20_NODE_1_12,VOG21261,L-alanyl-D-glutamate peptidase,"Viruses;dsDNA viruses, no RNA stage;Caudovirales"
2,AG-359-N20_NODE_1_14,VOG01003,REFSEQ hypothetical protein,"Viruses;dsDNA viruses, no RNA stage;Caudovirales"
3,AG-359-N20_NODE_1_16,VOG00072,"Terminase, large subunit gp2","Viruses;dsDNA viruses, no RNA stage;Caudovirales"
4,AG-359-N20_NODE_1_17,VOG03605,Probable portal protein,"Viruses;dsDNA viruses, no RNA stage;Caudovirales"
5,AG-359-N20_NODE_1_2,VOG00597,REFSEQ hypothetical protein,"Viruses;dsDNA viruses, no RNA stage;Caudovirales"
6,AG-359-N20_NODE_1_22,VOG19718,REFSEQ hypothetical protein,"Viruses;dsDNA viruses, no RNA stage;Caudoviral..."
7,AG-359-N20_NODE_1_23,VOG00325,Major capsid protein,"Viruses;dsDNA viruses, no RNA stage;Caudovirales"
8,AG-359-N20_NODE_1_25,VOG01520,Tail tubular protein gp11,"Viruses;dsDNA viruses, no RNA stage;Caudovirales"
9,AG-359-N20_NODE_1_26,VOG01653,REFSEQ hypothetical protein,"Viruses;dsDNA viruses, no RNA stage;Caudoviral..."


In [184]:
def hmm_vog_lca_per_contig(summary):
    best_guesses = []

    clca_count = summary.groupby(['contig','lca'])['orf'].count().reset_index()
    for c in clca_count['contig'].unique():
        subdf = clca_count[clca_count['contig'] == c]
        maxcount = 0
        for i, l in subdf.iterrows():
            if l['orf'] > maxcount:
                maxcount = l['orf']
                best_guess = l
            elif l['orf'] == maxcount:
                if len(l['lca'].split(";")) > len(best_guess['lca'].split(";")):
                    best_guess = l
                else:
                    continue
        best_guesses.append(best_guess)
    return pd.DataFrame(best_guesses)[['contig', 'lca','orf']].rename(columns={'orf':'hit_count', 'lca':'vog_lca'})

In [185]:
hmm_vog_lca_per_contig(vs)

,contig,vog_lca,hit_count
2,AG-359-N20_NODE_1,"Viruses;dsDNA viruses, no RNA stage;Caudovirales",17
9,AG-359-O06_NODE_1,"Viruses;dsDNA viruses, no RNA stage;Caudoviral...",5
11,AG-359-O06_NODE_2,"Viruses;dsDNA viruses, no RNA stage;Caudoviral...",3
13,AG-359-O06_NODE_3,"Viruses;dsDNA viruses, no RNA stage;Caudoviral...",1
16,AG-359-O19_NODE_1,"Viruses;dsDNA viruses, no RNA stage;Caudovirales",19
20,AG-359-O19_NODE_2,"Viruses;dsDNA viruses, no RNA stage;Caudovirales",3
22,AG-359-O19_NODE_3,"Viruses;dsDNA viruses, no RNA stage;Caudovirales",12
24,AG-359-O19_NODE_4,"Viruses;dsDNA viruses, no RNA stage;Caudovirales",4
26,AG-359-O19_NODE_5,"Viruses;dsDNA viruses, no RNA stage;Caudovirales",6


In [195]:
from scgc import safe_makedir
from viruscope import run_prodigal

def hmm_vog_summary(vhmm_tops, 
                vog_annotations = "/mnt/scgc/simon/simonsproject/jb_vs_test/viral_dbs/VOG/vog.annotations.tsv", 
                vog_lca = "/mnt/scgc/simon/simonsproject/jb_vs_test/viral_dbs/VOG/vog.lca.tsv"):
    vdf = vhmm_tops[['qseqid','sseqid']].rename(columns={'sseqid':'vog','qseqid':'orf'})
    voganns = pd.read_csv(vog_annotations, sep="\t")
    voganns.rename(columns={'#GroupName':'vog','ConsensusFunctionalDescription':'func_desc'}, inplace=True)
    voglca = pd.read_csv(vog_lca, skiprows=1, sep="\t", names=['vog','genomes_in_group_and_lca','genomes_total_in_lca','lca'])
    summary = pd.merge(vdf, 
                       voganns, 
                       on='vog', 
                       how='left').merge(voglca, 
                                         on='vog', 
                                         how='left')
    summary['contig'] = ["_".join(i.split("_")[:-1]) for i in summary['orf']]
    summary['orf_num'] = [int(i.split("_")[-1]) for i in summary['orf']]
    summary = summary.sort_values(by=['contig','orf_num']).reset_index()
    return summary[['orf','contig', 'vog','func_desc','lca']]

infasta = "../test_data/bats248_vspos_subset.fasta"
outdir = "../test_data/hmm_test"

In [187]:
name = op.basename(infasta).split(".")[0].split("_")[0]


In [190]:
proddir = safe_makedir(op.join(outdir, "prodigal"))
prots = run_prodigal(infasta, proddir)

In [191]:
prots

'../test_data/hmm_test/prodigal/bats248_proteins.fasta'

In [192]:
out_hmm = op.join(outdir, '{}_vs_vog_hmmer.tbl'.format(name))
orf_summary = op.join(outdir, '{}_vs_vog_orf_summary.csv'.format(name))
contig_summary = op.join(outdir, '{}_vs_vog_contig_summary.csv'.format(name))



In [197]:
df = hmmscan_besthits(import_hmmscan_out(out_hmm), max_eval=0.001)
vog_annotations = "/mnt/scgc/simon/simonsproject/jb_vs_test/viral_dbs/VOG/vog.annotations.tsv"
vog_lca = "/mnt/scgc/simon/simonsproject/jb_vs_test/viral_dbs/VOG/vog.lca.tsv"
vsum = hmm_vog_summary(df, vog_annotations=vog_annotations, vog_lca=vog_lca)

In [198]:
vsum

,orf,contig,vog,func_desc,lca


In [199]:
df

,sseqid,accession,qseqid,accession,evalue,score,bias,d-evalue,d-score,d-bias,exp,reg,clu,ov,env,dom,rep,inc,description of target


In [201]:
!cat {out_hmm}

#                                                                 --- full sequence ---- --- best 1 domain ---- --- domain number estimation ----
# target name        accession  query name             accession    E-value  score  bias   E-value  score  bias   exp reg clu  ov env dom rep inc description of target
#------------------- ----------   -------------------- ---------- --------- ------ ----- --------- ------ -----   --- --- --- --- --- --- --- --- ---------------------
#
# Program:         hmmscan
# Version:         3.1b2 (February 2015)
# Pipeline mode:   SCAN
# Query file:      notebooks/VCProfiler/test_data/hmm_test/prodigal/bats248_proteins.fasta
# Target file:     /mnt/scgc/simon/simonsproject/jb_vs_test/viral_dbs/VOG/hmm/vog_hmm
# Option settings: hmmscan --tblout notebooks/VCProfiler/test_data/hmm_test/bats248_vs_vog_hmmer.tbl --cpu 1 /mnt/scgc/simon/simonsproject/jb_vs_test/viral_dbs/VOG/hmm/vog_hmm notebooks/VCProfiler/test_data/hmm_test/prodigal/bats248_proteins.fasta